# Customer & Revenue Analysis using SQL

In this project, I use SQL on top of the online retail dataset to compute key business KPIs
such as total revenue, number of orders, and customer-level metrics.


In [27]:

import pandas as pd
import sqlite3

df = pd.read_csv("online_retail.csv")
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Preparing the data again for the SQL analysis

For the SQL analysis, I keep only valid sales transactions with a known customer,
positive quantity and price, and create a `TotalRevenue` metric per transaction.


In [28]:

df_sql = df.dropna(subset=["CustomerID", "Description"])
df_sql = df_sql[(df_sql["Quantity"] > 0) & (df_sql["UnitPrice"] > 0)]
df_sql["TotalRevenue"] = df_sql["Quantity"] * df_sql["UnitPrice"]
df_sql.shape


(394303, 9)

## Creating a SQLite Table

The cleaned data is loaded into an SQLite database.


In [29]:

conn = sqlite3.connect(":memory:")
df_sql.to_sql("sales", conn, index=False, if_exists="replace")


394303

## KPIs

First, I compute basic high-level KPIs:
- total number of rows (transaction lines),
- total number of distinct orders,
- total number of distinct customers,
- total revenue.


In [30]:
query_overall = """
SELECT
    COUNT(*) AS total_rows,
    COUNT(DISTINCT InvoiceNo) AS total_orders,
    COUNT(DISTINCT CustomerID) AS total_customers,
    SUM(TotalRevenue) AS total_revenue
FROM sales;
"""

overall_kpis = pd.read_sql(query_overall, conn)
overall_kpis


,total_rows,total_orders,total_customers,total_revenue
0,394303,18521,4336,8.689251e+06


## Average order value

To understand how much revenue each order generates on average,
I first aggregate revenue at the order level and then compute summary statistics.


In [31]:
query_aov = """
WITH order_revenue AS (
    SELECT
        InvoiceNo,
        SUM(TotalRevenue) AS order_revenue
    FROM sales
    GROUP BY InvoiceNo
)
SELECT
    COUNT(*) AS num_orders,
    AVG(order_revenue) AS avg_order_value,
    MIN(order_revenue) AS min_order_value,
    MAX(order_revenue) AS max_order_value
FROM order_revenue;
"""

aov_stats = pd.read_sql(query_aov, conn)
aov_stats


,num_orders,avg_order_value,min_order_value,max_order_value
0,18521,469.15667,0.38,77183.6


The average order value is around 469, but there is a very wide range between the smallest and largest orders
(minimum below 1 and a maximum above 77k in revenue per order).

This suggests a mix of very small purchases and large ones, which applies to a business
serving both individual and business customers.


## Top customers by revenue

This section identifies the customers who contribute the most to total revenue


In [32]:
query_top_customers = """
SELECT
    CustomerID,
    SUM(TotalRevenue) AS customer_revenue,
    COUNT(DISTINCT InvoiceNo) AS num_orders
FROM sales
GROUP BY CustomerID
ORDER BY customer_revenue DESC
LIMIT 10;
"""

top_customers = pd.read_sql(query_top_customers, conn)
top_customers


,CustomerID,customer_revenue,num_orders
0,14646.0,278788.58,73
1,18102.0,259657.30,60
2,17450.0,194550.79,46
3,14911.0,143260.39,201
4,12415.0,123146.21,21
5,14156.0,116403.28,55
6,17511.0,90927.50,31
7,16029.0,81024.84,63
8,12346.0,77183.60,1
9,16684.0,66018.68,28


There are several customers that account for substantial revenues, therefore representing high-value accounts. However, purchasing habits vary greatly, with some making frequent purchases that amount to high revenues, while others make only one purchase at a time that may amount to high revenues. This indication tells us that revenue is generated by different types of customers and that there is an opportunity to use customer segmentation for revenue driving purposes based on order size or frequency of their orders.


## Revenue concetration by customers

To assess revenue concentration, I calculate the share of total revenue generated by the top customers.


In [33]:
query_customer_concentration = """
WITH customer_revenue AS (
    SELECT
        CustomerID,
        SUM(TotalRevenue) AS revenue
    FROM sales
    GROUP BY CustomerID
),
total_rev AS (
    SELECT SUM(revenue) AS total_revenue FROM customer_revenue
)
SELECT
    SUM(revenue) / (SELECT total_revenue FROM total_rev) AS top_10_customer_share
FROM (
    SELECT revenue
    FROM customer_revenue
    ORDER BY revenue DESC
    LIMIT 10
);
"""

pd.read_sql(query_customer_concentration, conn)


,top_10_customer_share
0,0.164682


The top 10 customers account for about 16.5% of total revenue.
This shows moderate revenue concentration, with high-value customers playing an important role,
while the majority of revenue is still generated by a broader customer base.


## frequency of customers

This section examines how many customers place multiple orders,
giving us info on customer retention.


In [34]:
query_repeat_customers = """
SELECT
    COUNT(*) AS repeat_customers
FROM (
    SELECT
        CustomerID,
        COUNT(DISTINCT InvoiceNo) AS num_orders
    FROM sales
    GROUP BY CustomerID
    HAVING num_orders > 1
);
"""

pd.read_sql(query_repeat_customers, conn)


,repeat_customers
0,2843


Out of 4,336 unique customers, 2,843 have placed more than one order,
suggesting a high level of repeats on placing orders.

This indicates that the business benefits from a strong returning customer base.


## Final Thoughts

The purpose of this SQL analysis was to review online retail customer metrics and revenue records. The online retailing database sample consists of approximately 394k transaction lines, 18.5k orders, and 4.3k customers.

From this analysis, we found that high-value customers exist; the top 10 customers generate approximately 16.5% of total revenue. This represents moderate revenue concentration rather than a revenue dependence on a few accounts.

When comparing the amount of the actual order versus the order date, there is a wide variation in order amounts and dates — small orders (individual purchase) versus larger orders (bulk purchase). In fact, there are many transactions with greater than $77,000 in revenue, e.g., a purchase of an item could be 77 times its store price.

The analysis also shows that within our study group, the majority of customers have made repeat purchases. Approximately 65% of customers have made more than one purchase. This indicates that acquiring a customer is important, but retaining customers using customer retention methods is equally valuable.

The results from both customer segmentation,
order frequency and order value, would indicate that the use of segmentation strategies for targeting and/or engaging customers is beneficial in maximizing revenue.
